# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle
import glob
from keras.utils import to_categorical

Using TensorFlow backend.


# Load processed dataset
The data we load here are already shuffled and normalized.

In [2]:
data_directory = 'dataset/4. normalized-50/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

x_train: (604, 26)
y_train: (604, 2)
x_test: (604, 26)
y_test: (604, 2)


# Load auto encoders

In [3]:
autoencoder_1_path = 'results/34-autoencoder-1-01/model/2018-03-09-03-01-07/weights.hdf5'
autoencoder_2_path = 'results/37-autoencoder-2-01/model/2018-03-09-05-06-53/weights.hdf5'
weights_1 = load_model(autoencoder_1_path)
weights_1.summary()
weights_2 = load_model(autoencoder_2_path)
weights_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 26)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                270       
_________________________________________________________________
dropout_5 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 26)                286       
Total params: 556
Trainable params: 556
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_2 (Den

# Set up constants for weight transfer

In [4]:
autoencoder_1_layer_units = 10
autoencoder_1_layer_name = 'dense_9'
autoencoder_1_layer_dropout = 0

autoencoder_2_layer_units = 8
autoencoder_2_layer_name = 'dense_2'
autoencoder_2_layer_dropout = 0

# Tubable parameters

In [5]:
epochs = 8000
batch_size = 20
learning_rate = 0.0001
decay = 0.00001
loss = keras.losses.categorical_crossentropy

# Model for classification

In [6]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/36-fine-tune-multiclass-50-01/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"

def create_model(layers, dropout):
    global date
    global autoencoder_path
    x = Input(shape=(x_train.shape[1],))
    
    y = Dense(units = autoencoder_1_layer_units, activation='relu')(x)
    y = Dropout(autoencoder_1_layer_dropout)(y)
    
    y = Dense(units = autoencoder_2_layer_units, activation='relu')(y)
    y = Dropout(autoencoder_2_layer_dropout)(y)
    
    y = Dense(units = layers['layer_1'], activation='relu')(y)
    y = Dropout(dropout * 0.75)(y)
    
    if (layers['layer_2'] > 0):
        y = Dense(units = layers['layer_2'], activation='relu')(y)
        y = Dropout(dropout)(y)
    
    if (layers['layer_3'] > 0):
        y = Dense(units = layers['layer_3'], activation='relu')(y)
        y = Dropout(dropout)(y)
    
    y = Dense(units = y_train.shape[1], activation='softmax')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Layer 6': layers['layer_1'],
        'Layer 8': layers['layer_2'],
        'Layer 10': layers['layer_3'],
        'Dropout 6': dropout * 0.75,
        'Dropout 8': dropout,
        'Dropout 10': dropout,
        'Learning rate': learning_rate,
        'Decay': decay,
        'Batch size': batch_size,
        'Autoencoder 1': autoencoder_1_path,
        'Autoencoder 2': autoencoder_2_path,
        'Epochs': epochs,
        'Fine tune': True,
        'Loss': loss
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=loss, metrics=['accuracy'])
    
    # Set layer weights to pre trained autoencoder weights
    model.layers[1].set_weights(weights_1.get_layer(autoencoder_1_layer_name).get_weights())
    model.layers[3].set_weights(weights_2.get_layer(autoencoder_2_layer_name).get_weights())
    
    return model

# Prepare callbacks

In [7]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
        callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, verbose=0, **kwargs)

# Train

In [8]:
model = KerasClassifierTensorBoard(build_fn=create_model, epochs=epochs, batch_size=batch_size)

layers = [
    {'layer_1': 4, 'layer_2': 0, 'layer_3': 0},
    {'layer_1': 6, 'layer_2': 4, 'layer_3': 0},
    {'layer_1': 16, 'layer_2': 4, 'layer_3': 0},
    {'layer_1': 32, 'layer_2': 16, 'layer_3': 0},
    {'layer_1': 64, 'layer_2': 32, 'layer_3': 16},
    {'layer_1': 64, 'layer_2': 16, 'layer_3': 0},
    {'layer_1': 128, 'layer_2': 16, 'layer_3': 0},
    {'layer_1': 10, 'layer_2': 20, 'layer_3': 0}
]
param_grid = dict(
    layers=layers,
    dropout=[0.3, 0.5, 0]
)
    
grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_data=(x_test, y_test)))
grid_result = grid.fit(x_train, y_train)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[CV] layers={'layer_3': 0, 'layer_2': 0, 'layer_1': 4}, dropout=0.3 ..
 20/604 [..............................] - ETA: 0s[CV]  layers={'layer_3': 0, 'layer_2': 0, 'layer_1': 4}, dropout=0.3, total=23.8min
[CV] layers={'layer_3': 0, 'layer_2': 4, 'layer_1': 6}, dropout=0.3 ..


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 23.8min remaining:    0.0s


 20/604 [..............................] - ETA: 0s[CV]  layers={'layer_3': 0, 'layer_2': 4, 'layer_1': 6}, dropout=0.3, total=24.4min
[CV] layers={'layer_3': 0, 'layer_2': 4, 'layer_1': 16}, dropout=0.3 .
 20/604 [..............................] - ETA: 0s[CV]  layers={'layer_3': 0, 'layer_2': 4, 'layer_1': 16}, dropout=0.3, total=24.3min
[CV] layers={'layer_3': 0, 'layer_2': 16, 'layer_1': 32}, dropout=0.3 
 20/604 [..............................] - ETA: 0s[CV]  layers={'layer_3': 0, 'layer_2': 16, 'layer_1': 32}, dropout=0.3, total=24.6min
[CV] layers={'layer_3': 16, 'layer_2': 32, 'layer_1': 64}, dropout=0.3 
 20/604 [..............................] - ETA: 0s[CV]  layers={'layer_3': 16, 'layer_2': 32, 'layer_1': 64}, dropout=0.3, total=26.5min
[CV] layers={'layer_3': 0, 'layer_2': 16, 'layer_1': 64}, dropout=0.3 
 20/604 [..............................] - ETA: 0s[CV]  layers={'layer_3': 0, 'layer_2': 16, 'layer_1': 64}, dropout=0.3, total=24.7min
[CV] layers={'layer_3': 0, 'layer_2':

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 608.5min finished
